In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# initiate driver and get website
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.benzinga.com/analyst-stock-ratings')



[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 20.8MB/s]


In [2]:
# Handle popups

def popup_detect_main(row_num):
    try:
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                    f'//*[@id="__next"]/div[3]/div/div[1]/div[2]/div[1]/div/div[2]/div[2]/div/div[2]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{row_num}]/td[7]/div/span[2]'))).click()
    except:
        return True


def popup_detect_sub(urls,driver):
    try:
        urls.append(driver.current_url)
    except:
        return True




def find_popup_main(row_num):
    
    lst = [['Campaign__innerWrapper','esc'],['no-thanks','enter'],['call-to-action','esc'],['close-btn','enter']]
    for i in lst:
        #print(i[0],end=" ")
        try:
            popup = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.CLASS_NAME,i[0])))
            if i[1] == 'esc':
                popup.send_keys(Keys.ESCAPE)
            else:
                popup.send_keys(Keys.ENTER)
        except:
            try:
                popup = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="__next"]/div[3]/div/div[3]/div/div/div[1]'))).click()
            except:
                continue


def find_popup_sub():
    
    lst = [['Campaign__innerWrapper','esc'],['no-thanks','enter'],['call-to-action','esc'],['close-btn','enter']]
    for i in lst:
        #print(i[0],end=" ")
        try:
            popup = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.CLASS_NAME,i[0])))
            if i[1] == 'esc':
                popup.send_keys(Keys.ESCAPE)
            else:
                popup.send_keys(Keys.ENTER)
        except:
            try:
                popup = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="__next"]/div[3]/div/div[3]/div/div/div[1]'))).click()
            except:
                continue

In [3]:
# Ranges from row 2 to row 1001

# Get the analyst's page

urls = []
df = pd.DataFrame(columns=['analyst_url'])

for i in range(2,1001):
    
    
    driver.get('https://www.benzinga.com/analyst-stock-ratings')
    
    while(popup_detect_main(i)):
        find_popup_main(i)

    time.sleep(8)
    
    while(popup_detect_sub(urls,driver)):
        find_popup_sub()
        
    if i%50 ==0:
        print(str(i) + "th step")

    url_df = pd.DataFrame(urls,columns=['analyst_url'])
    df = pd.concat([df,url_df],ignore_index=True).drop_duplicates()
    df.to_csv('analysts_urls.csv')
       


50th step
100th step
150th step
200th step
250th step
300th step
350th step
400th step
450th step
500th step
550th step
600th step
650th step
700th step
750th step
800th step


In [40]:


cols = ['date','analyst','ticker','company','currentPrice','predPriceChange','signal']

df = pd.DataFrame(columns=['date','analyst','ticker','company','currentPrice','predPriceChange','signal'])

urls = pd.read_csv(r'analysts_urls.csv')['analyst_url']

for url in urls:

    driver.get(url)

    #

    last_row = int(WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            '//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[2]/div/div[4]/div/div/div'))).text) + 1 

    analyst = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="__next"]/div[3]/div/div/div[1]/div/div/div/div[3]/span'))).text


    for i in range(2,last_row+1):

        # Date
        date = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[1]'))).text

        # Ticker
        ticker = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[2]'))).text

        # Company Name
        compName = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[3]'))).text

        # Current Price
        currPrice = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[4]'))).text

        # up/downside
        priceChange = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[5]'))).text

        # Signal
        signal = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,
                                                                            f'//*[@id="__next"]/div[3]/div/div/div[2]/div[1]/div/div[2]/div[3]/div/div[1]/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{i}]/td[8]'))).text
        
        add_rating = pd.DataFrame([[date,analyst,ticker,compName,currPrice,priceChange,signal]],columns=cols)
        df = pd.concat([add_rating,df],ignore_index=True)

    




'https://www.benzinga.com/analyst-stock-ratings/analyst/5f7da386a442ba0001da775d'